<a href="https://colab.research.google.com/github/arvindjangra1996/testing_api/blob/master/API_TESTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependency

In [0]:
!pip install schemathesis==1.1.2

In [0]:
from schemathesis import runner, models, loaders
from datetime import timedelta
from hypothesis import Phase
from schemathesis.runner.events import BeforeExecution,AfterExecution

class Environments:
    Dev = {'apikey': 'YJgvtmHEWY39vxITA2fDW8cdrH6s4Epz9bYvlACE' ,'base_url': 'https://api.sentieo.com/v1'}
    Production = { 'apikey':'N4K02o4YFh1VDbsvxlnAa75V4UO0JLJk2Lwm4f1u' ,'base_url': 'https://pzeiwi7e75.execute-api.us-east-1.amazonaws.com/app2'}


In [0]:
#checks
def check_for_performance(response, case: models.Case):
    assert response.elapsed < timedelta(milliseconds=10000)
    
def check_for_response_status(response,case: models.Case):
    assert response.status_code == 200 or response.status_code == 204

def after_run(event,reports):
    if event.result.path not in reports:
        reports[event.result.path] = {}
    if event.result.method not in  reports[event.result.path]:
        reports[event.result.path][event.result.method] = {}   
    reports[event.result.path][event.result.method].update({'status':event.status.name })
    
    
additional_checks = [check_for_performance, check_for_response_status]
check_for_apis =  { '/documents/get':   [ check_for_performance ] }

In [0]:

github_repo = 'https://raw.githubusercontent.com/usernames/sentieodocs/'
github_branch_or_commit = 'monitorBranch'  # use branch or commit_id(example: e7e9fe5279aedaa3cf68406c308f89b7310c93d4 )
github_path = '/play-dev/sentieo.json'
github_token = '2a85e4e2cdc68408f0ca3690f84f84c914dd20d0'
endpoint = '/documents' # to run documents apis. None for running all.

reports = {}
runner_config = 'threadpool' | 'inprocess'
    
events = runner.prepare(
    schema_uri=json.dumps(schema),  #github_repo+github_branch+github_path,
    headers=api_headers,
    loader=loaders.from_file,
    base_url=gateway_url,
    hypothesis_deadline=10000,
    checks=additional_checks,
    validate_schema=False,
    endpoint=endpoint,
    hypothesis_phases=[Phase.explicit]
    #auth=github_auth
    )

for event in events:
    if isinstance(event,AfterExecution):
        after_run(event,reports)
        
print(reports)